# Habib Şako TÜBİTAK Proje 3 

In [4]:
import yfinance
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
# To get multi output in jupyternotebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Sektorlerin listesine bir web-scraping ile erisilmesi ve verilerin elde edilmesi

In [2]:
def sectors_names():
    url = "https://stockanalysis.com/stocks/industry/sectors/"
    response = requests.get(url)
    # web adresi başarılı şekilde cevap döndü mü?
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(str(soup.find_all("table")))[0]
    else:
        print(f"Error : Failed to fetch data from page {url}")
    return df

In [3]:
def fetch_data(sectors):
    url = f"https://stockanalysis.com/stocks/sector/{sectors}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(str(soup.find_all("table")))[0]
        df.drop(columns='No.', inplace = True)
    else:
        print("Error")

    return df
    

In [4]:
sectors = sectors_names()
sectors

C:\Users\habib\AppData\Local\Temp\ipykernel_22608\2852052015.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup.find_all("table")))[0]


,Sector Name,Stocks,Market Cap,Div. Yield,PE Ratio,Profit Margin,1D Change,1Y Change
0,Financials,1384,"9,687.22B",2.41%,14.70,17.75%,0.54%,11.61%
1,Healthcare,1217,"8,168.29B",0.43%,50.93,4.07%,1.32%,7.52%
2,Technology,788,17.72T,0.43%,44.88,13.16%,0.27%,14.87%
3,Industrials,652,"5,478.52B",1.11%,26.71,7.32%,0.55%,17.56%
4,Consumer Discretionary,577,"7,264.64B",0.65%,27.33,5.95%,0.31%,2.23%
5,Real Estate,264,"1,506.14B",4.13%,50.70,8.89%,-0.12%,6.92%
6,Materials,263,"2,069.59B",1.56%,19.61,8.68%,0.46%,2.26%
7,Communication Services,260,"5,361.34B",1.09%,28.01,10.40%,0.30%,2.10%
8,Energy,253,"3,646.11B",2.85%,8.01,12.42%,0.43%,18.22%
9,Consumer Staples,241,"4,035.49B",1.45%,29.74,4.72%,0.45%,12.15%


In [5]:
# fetch_data(sectors='energy').to_csv('./data/energy.csv')
# fetch_data(sectors='financials').to_csv('./data/financials.csv')
# fetch_data(sectors='healthcare').to_csv('./data/healthcare.csv')
# fetch_data(sectors='technology').to_csv('./data/technology.csv')
# fetch_data(sectors='industrials').to_csv('./data/industrials.csv')
# fetch_data(sectors='consumer-discretionary').to_csv('./data/consumer-discretionary.csv')
# fetch_data(sectors='materials').to_csv('./data/materials.csv')
# fetch_data(sectors='real-estate').to_csv('./data/real-estate.csv')
# fetch_data(sectors='communication-services').to_csv('./data/communication-services.csv')
# fetch_data(sectors='energy').to_csv('./data/energy.csv')
# fetch_data(sectors='consumer-staples').to_csv('./data/consumer-staples.csv')
# fetch_data(sectors='utilities').to_csv('./data/utilities.csv')


In [5]:
finance = pd.read_csv('data/financials.csv')
technology = pd.read_csv('data/technology.csv')
healthcare = pd.read_csv('data/healthcare.csv')

ticker_list_finance = finance.Symbol.to_list()
ticker_list_technology = technology.Symbol.to_list()
ticker_list_healthcare = healthcare.Symbol.to_list()


In [6]:
ticker_list_technology = [x for x in ticker_list_technology if not pd.isna(x)]

### 2005-01-01 yilindan itibaren aylik getirelerden olusan serilerin elde edilmesi

In [7]:
# SEÇİLEN SEKTÖRLERİN VERİLERİNİN ÇEKİLMESİ

# def get_ticker(ticker):
#     data = yfinance.download(ticker, start = "2005-01-01")
#     data.insert(0,"id", ticker)
#     data.reset_index(inplace = True)
#     return data

# def set_tickers(ticker_list):
#     merged_tickers = pd.DataFrame()
#     for ticker in ticker_list:
#         merged_tickers = pd.concat([merged_tickers, get_ticker(ticker) ])
#     return merged_tickers

def get_tickers(ticker_list):
    data = yfinance.download(ticker_list, start = "2005-01-01")
    data_close=data['Open'].resample('ME').last().pct_change(fill_method=None)+1
    return data_close


tickers_data_frame_f = get_tickers(ticker_list_finance) 
tickers_data_frame_t = get_tickers(ticker_list_technology) 
tickers_data_frame_h = get_tickers(ticker_list_healthcare) 


[**********************72%%*********             ]  723 of 1007 completedFailed to get ticker 'DYCQ' reason: Response ended prematurely
[*********************100%%**********************]  1007 of 1007 completed

8 Failed downloads:
['BNRE.A', 'CRD.B', 'LEGT', 'BRK.B', 'AGM.A', 'DISA', 'CRD.A', 'DYCQ']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  786 of 786 completed
[*********************100%%**********************]  1218 of 1218 completed

1 Failed download:
['BIO.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2005-01-01 -> 2024-03-17)')


In [8]:
# indirilen verilerin csv olarak kaydedilmesi

tickers_data_frame_f.to_csv('data/data_financials.csv')
tickers_data_frame_t.to_csv('data/data_technology.csv')
tickers_data_frame_h.to_csv('data/data_healthcare.csv')

In [9]:
data_f = pd.read_csv('data/data_financials.csv')
data_t = pd.read_csv('data/data_technology.csv')
data_h = pd.read_csv('data/data_healthcare.csv')

In [10]:
def nan_to_0 (df):
    df = df.fillna(0)
    return df

tickers_data_frame_f = nan_to_0(data_f)
tickers_data_frame_t = nan_to_0(data_t)
tickers_data_frame_h = nan_to_0(data_h)

tickers_data_frame_f.tail()

,Date,AACI,AACT,AAMC,AAME,AB,ABCB,ABL,ABTS,AC,...,WU,WULF,XFIN,XP,XPDB,XYF,YOTA,YRD,ZION,ZLS
226,2023-11-30,1.007477,1.006796,0.904078,1.133721,1.030851,1.141555,1.128927,1.081395,1.002390,...,1.030115,1.045045,1.000941,1.164500,1.002812,1.069652,1.001880,1.280543,1.168999,1.004726
227,2023-12-31,0.999072,1.006750,1.269939,1.256410,1.071848,1.249377,1.400000,1.225806,1.062165,...,1.045116,2.068966,1.006579,1.155537,1.003738,0.851163,0.970919,1.102473,1.231266,1.005644
228,2024-01-31,1.011142,1.005747,1.161836,1.110204,1.081212,0.935721,1.288820,0.947368,0.952394,...,1.054530,0.695833,1.013072,0.942846,1.014898,1.065574,1.025121,1.378205,0.955095,1.005613
229,2024-02-29,1.009183,1.002857,0.833680,1.003676,0.969081,0.933320,0.949398,0.910185,0.985004,...,1.066826,1.125749,1.000092,0.961758,0.993119,1.051282,1.036758,1.193023,0.950961,1.000000
230,2024-03-31,1.005460,1.003324,0.912718,0.992674,1.047844,0.984028,1.032149,0.762971,1.014328,...,1.022961,0.946808,1.001751,1.073181,1.792148,1.039024,1.000000,1.037037,1.009130,1.004651


###  3 buyuk sektor uzerinden getirilerin faktorleri(momentum gibi) hesaplanmasi

In [33]:
# 3 aylik momentum faktorlerinin hesaplanması

def get_rolling_ret(data, n):
    data_without_date = data.drop(columns = 'Date')
    rolling_result = data_without_date.rolling(n).apply(np.prod)
    # date sütununu kaybetmemek için tekrardan ekleme
    rolling_result['Date'] = data['Date']
    return rolling_result

calisilacak_veri_f = get_rolling_ret(tickers_data_frame_f, 3)
calisilacak_veri_t = get_rolling_ret(tickers_data_frame_t, 3)
calisilacak_veri_h = get_rolling_ret(tickers_data_frame_h, 3)
calisilacak_veri_f

,AACI,AACT,AAMC,AAME,AB,ABCB,ABL,ABTS,AC,ACAB,...,WULF,XFIN,XP,XPDB,XYF,YOTA,YRD,ZION,ZLS,Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-01-31
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-02-28
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005-03-31
3,0.000000,0.000000,0.000000,0.983553,1.017509,1.034789,0.000000,0.000000,0.000000,0.000000,...,0.631429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.037924,0.000000,2005-04-30
4,0.000000,0.000000,0.000000,0.996678,0.977508,0.998650,0.000000,0.000000,0.000000,0.000000,...,0.786517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.077251,0.000000,2005-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,1.023742,1.013685,0.656635,0.960591,0.956291,1.049135,0.757409,0.962733,0.925367,1.008563,...,0.555024,1.008531,0.941047,1.012299,1.000000,1.007561,1.052045,1.015876,1.013346,2023-11-30
227,1.006542,1.016553,1.020000,1.269430,1.045938,1.385738,1.414348,1.200000,0.981277,1.003795,...,1.904762,1.006579,1.167569,1.013207,0.894866,0.974576,1.283951,1.272682,1.012311,2023-12-31
228,1.017757,1.019417,1.333932,1.581395,1.194647,1.334556,2.036976,1.255814,1.014018,1.020755,...,1.504504,1.020696,1.268715,1.021556,0.970149,0.997180,1.945701,1.374714,1.016068,2024-01-31
229,1.019481,1.015429,1.230061,1.400000,1.123062,1.091115,1.713043,1.056989,0.996431,1.008490,...,1.620690,1.019831,1.047829,1.011682,0.953488,1.031895,1.812721,1.118307,1.011289,2024-02-29


In [63]:
# dataframe transpose almak ve nan olan iki sütunu silme işlemi

transpose_veri_f = calisilacak_veri_f.transpose()
transpose_veri_t = calisilacak_veri_t.transpose()
transpose_veri_h = calisilacak_veri_h.transpose()

transpose_veri_f = transpose_veri_f.drop(columns = [0,1,2], axis= 1)
transpose_veri_t = transpose_veri_t.drop(columns = [0,1,2], axis= 1)
transpose_veri_h = transpose_veri_h.drop(columns = [0,1,2], axis= 1)

transpose_veri_f
transpose_veri_t
transpose_veri_h

,3,4,5,6,7,8,9,10,11,12,...,221,222,223,224,225,226,227,228,229,230
AACI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.036706,1.037291,1.027317,1.023923,1.012299,1.023742,1.006542,1.017757,1.019481,1.025998
AACT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.011325,1.010795,1.013685,1.016553,1.019417,1.015429,1.011973
AAMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.863797,0.518974,0.086546,0.122798,0.142293,0.656635,1.02,1.333932,1.230061,0.884058
AAME,0.983553,0.996678,0.990099,1.003344,0.95,0.98,0.98,0.898246,0.918367,0.90136,...,0.787755,0.83945,0.959256,1.011236,0.950451,0.960591,1.26943,1.581395,1.4,1.106122
AB,1.017509,0.977508,1.003799,1.025898,0.98848,1.038958,1.179801,1.250652,1.196878,1.157665,...,0.897497,0.939448,0.907775,0.961799,0.909612,0.956291,1.045938,1.194647,1.123062,1.097911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOTA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.018464,1.028985,1.01244,1.013359,0.999061,1.007561,0.974576,0.99718,1.031895,1.062802
YRD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.068075,1.264151,1.331683,1.068132,0.824627,1.052045,1.283951,1.945701,1.812721,1.705128
ZION,1.037924,1.077251,1.070851,1.025982,0.991176,0.973506,1.033062,1.087902,1.06614,1.081499,...,0.913185,1.397043,1.315943,1.314036,0.815898,1.015876,1.272682,1.374714,1.118307,0.91655
ZLS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.015686,1.020488,1.018447,1.019305,1.011472,1.013346,1.012311,1.016068,1.011289,1.01029


,3,4,5,6,7,8,9,10,11,12,...,221,222,223,224,225,226,227,228,229,230
AAOI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.696833,3.479381,6.202703,1.840604,1.100741,0.962237,1.761167,2.117093,1.206038,0.65735
AAPL,0.937842,0.886313,0.883369,1.182751,1.179326,1.456398,1.350293,1.446364,1.340981,1.311165,...,1.177916,1.159369,1.061347,0.883854,0.870454,1.012403,1.126007,1.081238,0.95278,0.897732
ACIW,0.975989,0.974337,1.063931,1.2904,1.175593,1.130735,1.00972,1.086632,1.033752,1.2214,...,0.858784,0.915515,1.064445,0.973673,0.878396,1.101318,1.356383,1.47619,1.23074,1.040196
ACLS,0.831325,0.773256,0.939726,1.112721,0.887218,0.760933,0.629522,0.811864,0.913793,1.452874,...,1.375835,1.694674,1.219613,0.88938,0.635974,0.646786,0.7954,1.02,0.906582,0.829748
ACMR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.117949,1.40107,1.736894,1.384174,1.038168,0.947608,1.07926,1.266176,1.858173,1.38741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.027519,1.167047,0.72867,0.645924,0.506844,0.797447,1.127439,1.237654,1.166319,0.864792
ZM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.919285,1.194042,1.058096,1.030348,0.817723,0.954949,1.028167,1.077192,1.042754,0.917258
ZS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.252247,1.780022,1.151831,1.0635,0.989463,1.265812,1.423999,1.485097,1.224978,0.875745
ZUO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.110324,1.505777,0.8443,0.751139,0.631714,1.001098,1.140777,1.233468,0.885965,0.943617


,3,4,5,6,7,8,9,10,11,12,...,221,222,223,224,225,226,227,228,229,230
A,0.938489,1.000416,1.036937,1.264578,1.339442,1.422676,1.219893,1.108831,1.016489,1.059357,...,0.869235,0.899136,1.046685,0.929896,0.848896,1.055588,1.243337,1.258586,1.074804,1.060778
AADI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.944751,0.686856,0.7875,0.707602,0.821764,0.834921,0.417355,0.420091,0.374525,1.074257
ABBV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.845391,0.989809,1.065236,1.106361,0.943843,0.968903,1.039648,1.164471,1.236393,1.147835
ABCL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.856764,1.140118,0.781923,0.712074,0.531695,0.86422,1.241304,1.243309,1.072187,0.824869
ABEO,0.929368,0.817869,0.742972,0.624,0.298319,0.475676,0.647436,0.901408,0.590909,0.386139,...,1.429078,1.114007,1.433657,1.044665,1.116959,1.083522,1.190024,1.065445,1.4375,1.722555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZVRA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.927273,0.903704,1.005952,0.945098,0.918033,0.938856,1.358921,1.292411,1.415966,0.916031
ZVSA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.127461,0.372043,0.387978,0.528455,0.473988,0.457747,0.197802,0.198606,0.483516,0.867778
ZYME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.955752,0.888095,0.879708,0.733796,0.941019,1.225138,1.638801,1.54416,1.354002,1.040423
ZYXI,1.0,1.034483,1.956521,1.481477,1.833335,1.666666,1.700002,1.10909,0.626667,0.588234,...,0.799167,0.853147,0.820895,0.834202,0.909836,1.18961,1.36125,1.333333,1.480349,1.152433


In [209]:
# # tablolara yeni bir sütun ekleyerek sektörlerin baş harflerini yerleştirme
calisilacak_veri_f = transpose_veri_f.assign(Sector = 'Financials')
calisilacak_veri_t = transpose_veri_t.assign(Sector = 'Technology')
calisilacak_veri_h = transpose_veri_h.assign(Sector = 'Healthcare')

calisilacak_veri_f
calisilacak_veri_t
calisilacak_veri_h

# İndeks etiketi 'Date' olan satırı silme -- Dataframeleri birleştirirken 3 adet date satırı olmasın diye
calisilacak_veri_f = calisilacak_veri_f.drop('Date')
calisilacak_veri_h = calisilacak_veri_h.drop('Date')

# calisilacak_veri_t.drop(columns=['Tickets'], inplace=True) 
# calisilacak_veri_f.drop(columns=['Tickets'], inplace=True) 
# calisilacak_veri_h.drop(columns=['Tickets'], inplace=True) 

,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
AACI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.037291,1.027317,1.023923,1.012299,1.023742,1.006542,1.017757,1.019481,1.025998,Financials
AACT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.011325,1.010795,1.013685,1.016553,1.019417,1.015429,1.011973,Financials
AAMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.518974,0.086546,0.122798,0.142293,0.656635,1.02,1.333932,1.230061,0.884058,Financials
AAME,0.983553,0.996678,0.990099,1.003344,0.95,0.98,0.98,0.898246,0.918367,0.90136,...,0.83945,0.959256,1.011236,0.950451,0.960591,1.26943,1.581395,1.4,1.106122,Financials
AB,1.017509,0.977508,1.003799,1.025898,0.98848,1.038958,1.179801,1.250652,1.196878,1.157665,...,0.939448,0.907775,0.961799,0.909612,0.956291,1.045938,1.194647,1.123062,1.097911,Financials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOTA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.028985,1.01244,1.013359,0.999061,1.007561,0.974576,0.99718,1.031895,1.062802,Financials
YRD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.264151,1.331683,1.068132,0.824627,1.052045,1.283951,1.945701,1.812721,1.705128,Financials
ZION,1.037924,1.077251,1.070851,1.025982,0.991176,0.973506,1.033062,1.087902,1.06614,1.081499,...,1.397043,1.315943,1.314036,0.815898,1.015876,1.272682,1.374714,1.118307,0.91655,Financials
ZLS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.020488,1.018447,1.019305,1.011472,1.013346,1.012311,1.016068,1.011289,1.01029,Financials


,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
AAOI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.479381,6.202703,1.840604,1.100741,0.962237,1.761167,2.117093,1.206038,0.65735,Technology
AAPL,0.937842,0.886313,0.883369,1.182751,1.179326,1.456398,1.350293,1.446364,1.340981,1.311165,...,1.159369,1.061347,0.883854,0.870454,1.012403,1.126007,1.081238,0.95278,0.897732,Technology
ACIW,0.975989,0.974337,1.063931,1.2904,1.175593,1.130735,1.00972,1.086632,1.033752,1.2214,...,0.915515,1.064445,0.973673,0.878396,1.101318,1.356383,1.47619,1.23074,1.040196,Technology
ACLS,0.831325,0.773256,0.939726,1.112721,0.887218,0.760933,0.629522,0.811864,0.913793,1.452874,...,1.694674,1.219613,0.88938,0.635974,0.646786,0.7954,1.02,0.906582,0.829748,Technology
ACMR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.40107,1.736894,1.384174,1.038168,0.947608,1.07926,1.266176,1.858173,1.38741,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.167047,0.72867,0.645924,0.506844,0.797447,1.127439,1.237654,1.166319,0.864792,Technology
ZM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.194042,1.058096,1.030348,0.817723,0.954949,1.028167,1.077192,1.042754,0.917258,Technology
ZS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.780022,1.151831,1.0635,0.989463,1.265812,1.423999,1.485097,1.224978,0.875745,Technology
ZUO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.505777,0.8443,0.751139,0.631714,1.001098,1.140777,1.233468,0.885965,0.943617,Technology


,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
A,0.938489,1.000416,1.036937,1.264578,1.339442,1.422676,1.219893,1.108831,1.016489,1.059357,...,0.899136,1.046685,0.929896,0.848896,1.055588,1.243337,1.258586,1.074804,1.060778,Healthcare
AADI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.686856,0.7875,0.707602,0.821764,0.834921,0.417355,0.420091,0.374525,1.074257,Healthcare
ABBV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.989809,1.065236,1.106361,0.943843,0.968903,1.039648,1.164471,1.236393,1.147835,Healthcare
ABCL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.140118,0.781923,0.712074,0.531695,0.86422,1.241304,1.243309,1.072187,0.824869,Healthcare
ABEO,0.929368,0.817869,0.742972,0.624,0.298319,0.475676,0.647436,0.901408,0.590909,0.386139,...,1.114007,1.433657,1.044665,1.116959,1.083522,1.190024,1.065445,1.4375,1.722555,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZVRA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.903704,1.005952,0.945098,0.918033,0.938856,1.358921,1.292411,1.415966,0.916031,Healthcare
ZVSA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.372043,0.387978,0.528455,0.473988,0.457747,0.197802,0.198606,0.483516,0.867778,Healthcare
ZYME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.888095,0.879708,0.733796,0.941019,1.225138,1.638801,1.54416,1.354002,1.040423,Healthcare
ZYXI,1.0,1.034483,1.956521,1.481477,1.833335,1.666666,1.700002,1.10909,0.626667,0.588234,...,0.853147,0.820895,0.834202,0.909836,1.18961,1.36125,1.333333,1.480349,1.152433,Healthcare


In [210]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(calisilacak_veri_f)
# profile.to_file("data/profile_f.html")

In [211]:
# daha önceden momentum oluşturmak için nan değerlerini 0 çevirmişitim. Boş değerleri tespit etmek ve doldurmak için tekrar nan çeviriyorum.
def zero_to_nan(df):
    df.replace(0, np.nan, inplace = True)
    return df

zero_to_nan(calisilacak_veri_f).head()
zero_to_nan(calisilacak_veri_t).head()
zero_to_nan(calisilacak_veri_h).head()

C:\Users\habib\AppData\Local\Temp\ipykernel_22608\3128883133.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(0, np.nan, inplace = True)


,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
AACI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.037291,1.027317,1.023923,1.012299,1.023742,1.006542,1.017757,1.019481,1.025998,Financials
AACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.011325,1.010795,1.013685,1.016553,1.019417,1.015429,1.011973,Financials
AAMC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.518974,0.086546,0.122798,0.142293,0.656635,1.020000,1.333932,1.230061,0.884058,Financials
AAME,0.983553,0.996678,0.990099,1.003344,0.95000,0.980000,0.980000,0.898246,0.918367,0.901360,...,0.839450,0.959256,1.011236,0.950451,0.960591,1.269430,1.581395,1.400000,1.106122,Financials
AB,1.017509,0.977508,1.003799,1.025898,0.98848,1.038958,1.179801,1.250652,1.196878,1.157665,...,0.939448,0.907775,0.961799,0.909612,0.956291,1.045938,1.194647,1.123062,1.097911,Financials


,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
AAOI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.479381,6.202703,1.840604,1.100741,0.962237,1.761167,2.117093,1.206038,0.65735,Technology
AAPL,0.937842,0.886313,0.883369,1.182751,1.179326,1.456398,1.350293,1.446364,1.340981,1.311165,...,1.159369,1.061347,0.883854,0.870454,1.012403,1.126007,1.081238,0.95278,0.897732,Technology
ACIW,0.975989,0.974337,1.063931,1.2904,1.175593,1.130735,1.00972,1.086632,1.033752,1.2214,...,0.915515,1.064445,0.973673,0.878396,1.101318,1.356383,1.47619,1.23074,1.040196,Technology
ACLS,0.831325,0.773256,0.939726,1.112721,0.887218,0.760933,0.629522,0.811864,0.913793,1.452874,...,1.694674,1.219613,0.88938,0.635974,0.646786,0.7954,1.02,0.906582,0.829748,Technology
ACMR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.40107,1.736894,1.384174,1.038168,0.947608,1.07926,1.266176,1.858173,1.38741,Technology


C:\Users\habib\AppData\Local\Temp\ipykernel_22608\3128883133.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(0, np.nan, inplace = True)


,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
A,0.938489,1.000416,1.036937,1.264578,1.339442,1.422676,1.219893,1.108831,1.016489,1.059357,...,0.899136,1.046685,0.929896,0.848896,1.055588,1.243337,1.258586,1.074804,1.060778,Healthcare
AADI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.686856,0.787500,0.707602,0.821764,0.834921,0.417355,0.420091,0.374525,1.074257,Healthcare
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.989809,1.065236,1.106361,0.943843,0.968903,1.039648,1.164471,1.236393,1.147835,Healthcare
ABCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.140118,0.781923,0.712074,0.531695,0.864220,1.241304,1.243309,1.072187,0.824869,Healthcare
ABEO,0.929368,0.817869,0.742972,0.624000,0.298319,0.475676,0.647436,0.901408,0.590909,0.386139,...,1.114007,1.433657,1.044665,1.116959,1.083522,1.190024,1.065445,1.437500,1.722555,Healthcare


### Imputing işlemi - boş değerleri doldurma

In [212]:
from sklearn.impute import SimpleImputer

In [13]:
# Eksik değerleri ortalama ile doldurma
def impute(calisilacak_veri):
        
    if 'Sector' in calisilacak_veri.columns:
        df = calisilacak_veri.drop(columns=['Sector'])
        df = df.drop(df.index[-1])
    else:
        df = calisilacak_veri
    imputer = SimpleImputer(strategy='mean')  
    imputed_data = imputer.fit_transform(df)
    imputed_data
    # Doldurulmuş veriyi DataFrame'e dönüştürme
    imputed_df = pd.DataFrame(imputed_data, columns=df.columns)
    return imputed_df

# calisilacak_veri_f = impute(calisilacak_veri_f)
# calisilacak_veri_t = impute(calisilacak_veri_t)
# calisilacak_veri_h = impute(calisilacak_veri_h)

In [214]:
# # tablolara yeni bir sütun ekleyerek sektörlerin baş harflerini yerleştirme
calisilacak_veri_f = calisilacak_veri_f.assign(Sector = 'Financials')
calisilacak_veri_t = calisilacak_veri_t.assign(Sector = 'Technology')
calisilacak_veri_h = calisilacak_veri_h.assign(Sector = 'Healthcare')
calisilacak_veri_f

,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
0,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.037291,1.027317,1.023923,1.012299,1.023742,1.006542,1.017757,1.019481,1.025998,Financials
1,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.109965,1.092169,1.011325,1.010795,1.013685,1.016553,1.019417,1.015429,1.011973,Financials
2,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,0.518974,0.086546,0.122798,0.142293,0.656635,1.020000,1.333932,1.230061,0.884058,Financials
3,0.983553,0.996678,0.990099,1.003344,0.950000,0.980000,0.980000,0.898246,0.918367,0.901360,...,0.839450,0.959256,1.011236,0.950451,0.960591,1.269430,1.581395,1.400000,1.106122,Financials
4,1.017509,0.977508,1.003799,1.025898,0.988480,1.038958,1.179801,1.250652,1.196878,1.157665,...,0.939448,0.907775,0.961799,0.909612,0.956291,1.045938,1.194647,1.123062,1.097911,Financials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.015519,1.020270,1.019231,1.019102,1.012299,1.013207,1.021556,1.011682,1.806331,Financials
1002,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.436782,1.283582,0.931663,0.804000,1.000000,0.894866,0.970149,0.953488,1.163934,Financials
1003,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.028985,1.012440,1.013359,0.999061,1.007561,0.974576,0.997180,1.031895,1.062802,Financials
1004,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.264151,1.331683,1.068132,0.824627,1.052045,1.283951,1.945701,1.812721,1.705128,Financials


In [287]:
# tech, health ve finance dataframelerinin birleştirilmesi
calisilacak_veri = pd.concat([calisilacak_veri_f,calisilacak_veri_h,calisilacak_veri_t],axis=0, ignore_index = False)
calisilacak_veri

,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
0,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.037291,1.027317,1.023923,1.012299,1.023742,1.006542,1.017757,1.019481,1.025998,Financials
1,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.109965,1.092169,1.011325,1.010795,1.013685,1.016553,1.019417,1.015429,1.011973,Financials
2,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,0.518974,0.086546,0.122798,0.142293,0.656635,1.020000,1.333932,1.230061,0.884058,Financials
3,0.983553,0.996678,0.990099,1.003344,0.950000,0.980000,0.980000,0.898246,0.918367,0.901360,...,0.839450,0.959256,1.011236,0.950451,0.960591,1.269430,1.581395,1.400000,1.106122,Financials
4,1.017509,0.977508,1.003799,1.025898,0.988480,1.038958,1.179801,1.250652,1.196878,1.157665,...,0.939448,0.907775,0.961799,0.909612,0.956291,1.045938,1.194647,1.123062,1.097911,Financials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.194185,1.009709,0.879000,0.560870,0.618269,0.989761,1.361240,1.726283,1.298851,Technology
782,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.167047,0.728670,0.645924,0.506844,0.797447,1.127439,1.237654,1.166319,0.864792,Technology
783,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.194042,1.058096,1.030348,0.817723,0.954949,1.028167,1.077192,1.042754,0.917258,Technology
784,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.780022,1.151831,1.063500,0.989463,1.265812,1.423999,1.485097,1.224978,0.875745,Technology


In [288]:
# birleştirilmiş ve ön işlemden geçmiş verinin kaydedilmesi
calisilacak_veri.to_csv('data/calisilacak_veri.csv')
# calisilacak_veri.describe()

# tsfresh için date column ekleme
# calisilacak_veri = calisilacak_veri.transpose()

calisilacak_veri 

,3,4,5,6,7,8,9,10,11,12,...,222,223,224,225,226,227,228,229,230,Sector
0,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.037291,1.027317,1.023923,1.012299,1.023742,1.006542,1.017757,1.019481,1.025998,Financials
1,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.109965,1.092169,1.011325,1.010795,1.013685,1.016553,1.019417,1.015429,1.011973,Financials
2,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,0.518974,0.086546,0.122798,0.142293,0.656635,1.020000,1.333932,1.230061,0.884058,Financials
3,0.983553,0.996678,0.990099,1.003344,0.950000,0.980000,0.980000,0.898246,0.918367,0.901360,...,0.839450,0.959256,1.011236,0.950451,0.960591,1.269430,1.581395,1.400000,1.106122,Financials
4,1.017509,0.977508,1.003799,1.025898,0.988480,1.038958,1.179801,1.250652,1.196878,1.157665,...,0.939448,0.907775,0.961799,0.909612,0.956291,1.045938,1.194647,1.123062,1.097911,Financials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.194185,1.009709,0.879000,0.560870,0.618269,0.989761,1.361240,1.726283,1.298851,Technology
782,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.167047,0.728670,0.645924,0.506844,0.797447,1.127439,1.237654,1.166319,0.864792,Technology
783,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.194042,1.058096,1.030348,0.817723,0.954949,1.028167,1.077192,1.042754,0.917258,Technology
784,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.780022,1.151831,1.063500,0.989463,1.265812,1.423999,1.485097,1.224978,0.875745,Technology


In [290]:
# tarihler = pd.date_range(start='2005-01-01', periods=229, freq='M')
# calisilacak_veri['Date'] = tarihler
# calisilacak_veri

### TSFresh ile özellik çıkarımı

In [12]:
from tsfresh import extract_features

In [279]:
###  4 farklı sütunu kullanarak özellik çıkarma
# extracted_features = extract_features(calisilacak_veri , column_id = [50, 130, 180, 220], column_sort ='Date')

### MODEL (LSTM) eğitilmesi ve Test

In [5]:
import onnx
import onnxruntime as ort
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


##### unhashable type: 'list' çözemediğm için LSTM kullanamadım.#######

# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# from keras.utils import to_categorical


from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

### Modeller

In [327]:
# modelleri tanımlama
models = [
    RandomForestClassifier(random_state=42),
    XGBClassifier(random_state=42)
]

### Label Encoding - Sektörleri sayısal değerlere dönüştürme

In [329]:
# label_encoder = LabelEncoder()
# calisilacak_veri['Encoded_Sector'] = label_encoder.fit_transform(calisilacak_veri['Sector'])

# # Dönüştürülmüş veriyi görüntüleme
# print(calisilacak_veri[['Sector', 'Encoded_Sector']].tail())

# ### financials ---> 0
# ### healthcare ---> 1
# ### technology ---> 2

# calisilacak_veri.drop(columns=['Sector'], inplace=True)

### Verinin Bölünmesi

In [433]:
X = calisilacak_veri.drop(columns='Encoded_Sector')
Y = calisilacak_veri['Encoded_Sector']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.27 , random_state = 42 )
X.columns = X.columns.astype(str)

In [434]:

# cv scores değerine göre en iyi modelin seçilmesi
for model in models:
    cv_scores = cross_val_score(model, X_train, Y_train, cv=5)
    print(f"Model: {model.__class__.__name__}, Mean Accuracy: {cv_scores.mean()}")

Model: RandomForestClassifier, Mean Accuracy: 0.9312383516255954
Model: XGBClassifier, Mean Accuracy: 0.9376206253882791


### En iyi Model Seçilmesi

In [435]:
best_model = max(models, key=lambda model: cross_val_score(model, X_train, Y_train, cv=5).mean())
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### Eğitim

In [436]:
best_model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### Test

In [437]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
accuracy
X_test

0.9360393603936039

,3,4,5,6,7,8,9,10,11,12,...,221,222,223,224,225,226,227,228,229,230
744,0.957285,0.980101,1.030960,1.102467,1.058030,1.046060,1.003806,1.079892,1.074298,1.122412,...,1.015152,1.019700,1.017724,1.024254,1.007360,1.008249,1.024590,1.010046,1.006364,0.976889
678,0.846512,0.902622,0.837670,1.050824,0.950208,1.026627,1.015686,1.228530,1.253602,1.296010,...,1.105088,1.172222,1.152762,0.863530,0.712085,0.853602,1.188563,1.203411,1.009679,0.763329
67,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.112172,1.303469,1.308268,0.876575,0.760243,0.897222,1.093737,2.752045,0.295455,0.300000
265,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.388571,2.171642,1.701389,0.835391,0.872852,1.346939,1.689655,0.755906,0.630303,0.548105
942,0.941233,1.024248,1.050000,1.060547,1.073238,1.093091,1.092594,1.117453,1.079502,1.092780,...,1.039322,1.151251,1.067684,0.998246,0.911833,1.140624,1.268478,1.143423,1.002578,0.951408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,0.933386,0.960419,1.049119,1.148939,1.081892,1.068976,0.992817,1.000669,1.062454,1.214306,...,1.096491,0.719512,0.323871,0.391200,0.393220,0.517928,0.971370,0.482759,0.263846,0.111579
20,0.908756,0.975924,1.013853,1.157504,1.039549,1.065918,0.987821,1.071959,1.054708,1.190036,...,1.060099,1.072782,0.987996,0.991264,0.988848,1.000000,0.159385,0.141917,0.142991,6.323529
812,0.933386,0.960419,1.049119,1.148939,1.081892,1.068976,0.992817,1.000669,1.062454,1.214306,...,2.602305,1.805845,1.963249,1.367663,1.536416,1.374384,1.136032,0.981189,0.888889,0.823236
406,0.933386,0.960419,1.049119,1.148939,1.081892,1.068976,0.992817,1.000669,1.062454,1.214306,...,0.726724,1.029870,0.870114,0.860024,0.863808,1.028985,0.827586,1.226277,1.936620,2.283333


### Modelin Kaydedilmesi

In [6]:
import joblib

# joblib.dump(model, "model.joblib")  # Modeli joblib formatında kaydetme

# joblib formatındaki modeli yeniden yükleme
loaded_model = joblib.load("model.joblib")


### Test için başka sektörden veri çekilmesi

In [68]:
utilities_data = pd.read_csv('data/utilities.csv')
utilities_data_tickers = utilities_data['Symbol'].to_list()

In [69]:
data = yfinance.download(utilities_data_tickers, start = "2005-01-01")
data_close=data['Open'].resample('ME').last().pct_change(fill_method=None)+1
data_close.transpose()

[*********************100%%**********************]  108 of 108 completed


Date,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,...,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31
Ticker,,,,,,,,,,,,,,,,,,,,,
ADN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.592233,1.427869,0.671642,0.675214,0.936709,1.162162,0.511628,0.850000,1.016043,1.000000
AEE,NaN,1.031068,0.951205,1.048437,1.058869,1.025773,1.001256,0.979028,0.986635,0.967155,...,1.008715,1.063688,0.935252,0.931762,1.004527,1.023065,0.932495,0.982771,1.014845,0.996517
AEP,NaN,0.943583,1.009567,1.036423,1.014571,1.027316,1.077303,0.941985,1.069962,0.957586,...,1.020022,1.023100,0.932148,0.944277,1.006247,1.047021,1.022455,0.975817,1.062081,0.976190
AES,NaN,1.200143,0.963073,0.993197,0.904110,1.136364,0.973939,0.967019,1.039254,0.965944,...,1.038850,1.056338,0.842759,0.824332,0.986102,1.160403,1.109312,0.883212,0.900826,0.982962
AGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.018458,1.006397,0.920816,0.868852,0.968553,1.058442,1.050371,0.951430,1.009047,1.139289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WAVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.738562,1.092920,0.927125,1.165939,0.885019,0.732120,0.687861,1.277311,0.861842,0.946565
WEC,NaN,1.030910,1.010565,1.000283,1.026836,1.074278,1.030730,0.957764,1.042283,0.941513,...,1.008540,1.035702,0.942548,0.944790,1.039578,0.992601,1.006493,0.973360,0.951890,1.041258
WTRG,NaN,1.021895,1.010301,1.084421,1.039488,1.071997,1.080324,1.036551,1.139240,0.879630,...,0.980870,1.070162,0.880237,0.926593,0.970110,1.061621,1.056354,0.973860,0.962750,1.024751


In [70]:
from sklearn.impute import SimpleImputer

In [71]:
def impute(calisilacak_veri):
    df = calisilacak_veri
    imputer = SimpleImputer(strategy='mean', missing_values=np.nan)  # Eksik değerleri np.nan olarak tanımla
    imputed_data = imputer.fit_transform(df)
    imputed_df = pd.DataFrame(imputed_data, columns=df.columns)
    return imputed_df


In [72]:
# boş değerleri doldurma
data_close = impute(data_close)
data_close


Ticker,ADN,AEE,AEP,AES,AGR,ALCE,ALE,AMPS,AQN,ARIS,...,UGI,UTL,VGAS,VIA,VST,WAVE,WEC,WTRG,XEL,YORW
0,0.959402,1.002928,1.004962,1.004164,1.000713,0.951260,1.002942,0.993421,1.004900,1.017374,...,1.004249,1.004153,1.014738,1.001119,1.019374,0.956468,1.007887,1.005661,1.005584,1.006581
1,0.959402,1.031068,0.943583,1.200143,1.000713,0.951260,0.967944,0.993421,1.004900,1.017374,...,1.066346,0.985185,1.014738,1.001119,1.019374,0.956468,1.030910,1.021895,0.994475,1.001054
2,0.959402,0.951205,1.009567,0.963073,1.000713,0.951260,1.040837,0.993421,1.004900,1.017374,...,1.013977,0.971805,1.014738,1.001119,1.019374,0.956468,1.010565,1.010301,0.945556,1.005789
3,0.959402,1.048437,1.036423,0.993197,1.000713,0.951260,0.977512,0.993421,1.004900,1.017374,...,1.111383,0.994197,1.014738,1.001119,1.019374,0.956468,1.000283,1.084421,1.007050,1.086866
4,0.959402,1.058869,1.014571,0.904110,1.000713,0.951260,1.161527,0.993421,1.004900,1.017374,...,1.076215,1.056420,1.014738,1.001119,1.019374,0.956468,1.026836,1.039488,1.076429,0.990852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,1.162162,1.023065,1.047021,1.160403,1.058442,0.891612,1.036592,0.990291,1.175676,1.014925,...,1.067186,1.051416,0.731407,1.664260,1.067155,0.732120,0.992601,1.061621,1.019319,1.069062
227,0.511628,0.932495,1.022455,1.109312,1.050371,0.147992,1.107350,1.364706,1.029557,1.062500,...,1.124088,1.088479,0.778770,1.020607,1.108696,0.687861,1.006493,1.056354,1.025603,1.026523
228,0.850000,0.982771,0.975817,0.883212,0.951430,0.707143,0.980873,0.775862,0.971292,1.006920,...,0.921266,0.936227,1.177768,1.130712,1.066821,1.277311,0.973360,0.973860,0.976009,0.917882
229,1.016043,1.014845,1.062081,0.900826,1.009047,0.363636,0.942833,1.261111,0.957307,1.237113,...,1.085903,1.028467,1.581273,1.016917,1.286094,0.861842,0.951890,0.962750,0.952500,0.986901


In [82]:
# tüm veriyi test datası olarak verme
X = data_close.reset_index(drop=True)
X.columns = X.columns.astype(str)

In [80]:
# # Mevcut sütun isimlerini alalım
# eski_sutunlar = data_close.columns

# # Yeni sütun isimlerini oluşturalım
# yeni_sutunlar = [str(i) for i in range(3, 3+len(eski_sutunlar))]

# # DataFrame'in sütun isimlerini güncelleyelim
# data_close.columns = yeni_sutunlar
# data_close.drop

### test sektörün modele verilemsi

In [81]:

y_pred = loaded_model.predict(X)
y_pred

ValueError: feature_names mismatch: ['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '15